In [1]:
!pip install pymysql
!pip install cryptography

In [3]:
# Importing the pymysql library, which allows Python to interact with MySQL databases
import pymysql

# Importing the pandas library for data manipulation and analysis
import pandas as pd

# Importing the numpy library for numerical operations and handling arrays
import numpy as np


In [5]:
# Loading the "Account" sheet from the provided Excel file into a Pandas DataFrame
account = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Account")

# Loading the "Customer" sheet from the provided Excel file into a Pandas DataFrame
customer = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Customer")

# Loading the "Betting" sheet from the provided Excel file into a Pandas DataFrame
betting = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Betting")

# Loading the "Product" sheet from the provided Excel file into a Pandas DataFrame
product = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Product")

# Loading the "Student_School" sheet from the provided Excel file into a Pandas DataFrame
# Note: The 'header=1' argument indicates that the second row (index 1) of the sheet contains the column names
student_school = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Student_School", header=1)

In [7]:
# Removing duplicates from all tables to ensure clean data

# Remove duplicate rows in the account table
account = account.drop_duplicates()

# Remove duplicate rows in the customer table
customer = customer.drop_duplicates()

# Remove duplicate rows in the betting table
betting = betting.drop_duplicates()

# Remove duplicate rows in the product table
product = product.drop_duplicates()

# Remove duplicate rows in the student_school table (to extract student and school later)
student_school = student_school.drop_duplicates()


* Let's splits the student_school DataFrame into two subsets: one for student-related data (student) and another for school-related data (school).
* The .iloc method is used for selecting rows and columns by their numerical indices.

In [9]:
# Extracting subsets of Student and School data

# Extract the first five columns for the student table and remove any completely empty rows
student = student_school.iloc[1:, :5].dropna(how="all")  # Skip the first row (assumes formatting issues like dashes)

# Extract the last three columns for the school table and remove any completely empty rows
school = student_school.iloc[1:, -3:].dropna(how="all")  # Skip the first row (assumes formatting issues like dashes)

# Printing the cleaned DataFrames for verification
print("Student DataFrame:")
print(student)
print("\nSchool DataFrame:")
print(school)


Student DataFrame:
  student_id  student_name           city school_id  GPA
1       1001  Peter Brebec       New York         1    4
2       1002   John Goorgy  San Francisco         2  3.1
3       2003    Brad Smith       New York         3  2.9
4       1004  Fabian Johns         Boston         5  2.1
5       1005  Brad Cameron       Stanford         1  2.3
6       1006   Geoff Firby         Boston         5  1.2
7       1007   Johnny Blue      New Haven         2  3.8
8       1008   Johse Brook          Miami         2  3.4

School DataFrame:
  school_id.1                school_name         city.1
1           1                   Stanford       Stanford
2           2   University of California  San Francisco
3           3         Harvard University       New York
4           4                        MIT         Boston
5           5                       Yale      New Haven
6           6  University of Westminster         London
7           7        Corvinus University       Budapest


In [13]:
# Ensure all column names are strings
betting.columns = betting.columns.astype(str)

# Remove completely empty rows
betting = betting.dropna(how="all", axis=0)

# Drop columns that are completely empty or unnamed
columns_to_drop = betting.columns[betting.isnull().all()].tolist()  # Empty columns
betting = betting.drop(columns=columns_to_drop, errors='ignore')

# Explicitly drop the column with a 'NaN' or 'nan' header if it exists
if 'NaN' in betting.columns or 'nan' in betting.columns:
    betting = betting.drop(columns=['NaN', 'nan'], errors='ignore')

# Replace all NaN with None in the entire DataFrame
betting = betting.where(pd.notna(betting), None)

# Ensure 'AccountNo.1' and 'AccountNo.2' values are either None or valid in the account table
betting[['AccountNo.1', 'AccountNo.2']] = betting[['AccountNo.1', 'AccountNo.2']].apply(lambda col: col.apply(lambda x: None if pd.isna(x) else x))

# Validate 'AccountNo.1' and 'AccountNo.2' against the account table
betting = betting[
    betting['AccountNo'].notnull() &
    ((betting['AccountNo.1'].isnull()) | (betting['AccountNo.1'].isin(account['AccountNo']))) &
    ((betting['AccountNo.2'].isnull()) | (betting['AccountNo.2'].isin(account['AccountNo'])))
]

# Print the cleaned betting table for verification
print("Cleaned Betting DataFrame:")
print(betting)


Cleaned Betting DataFrame:
       AccountNo    BetDate     ClassId  CategoryId Source  BetCount  Bet_Amt  \
0     01196ZZ    2012-11-24  22512_XFER       22512      I       189  29364.0   
1     00357DG    2012-11-27  22512_XFER       22512      I      1415  19835.0   
2     00357DG    2012-11-24  22512_XFER       22512      I      2026  19280.8   
3     01196ZZ    2012-12-19  22512_XFER       22512      I       151  17301.0   
4     01284UW    2012-10-21          46           5      I        25  16850.0   
...          ...        ...         ...         ...    ...       ...      ...   
1015  01196ZZ    2012-12-19  22522_SALE       22522      I         0      0.0   
1016  01284UW    2012-10-15         274           5      I         0      0.0   
1017  01284UW    2012-11-05         424          17      I         1      0.0   
1018  01284UW    2012-12-21          46           5      I         0      0.0   
1019  01148BP    2012-10-04           2           9      I         2   -300.0   



In [15]:
# Establishing a connection to a MySQL database using pymysql.
# Replace 'root', 'MYPASSWORD', 'localhost', and 'sakila' with the appropriate credentials and database details.

cnx = pymysql.connect(
    user='root',                 # Username for the MySQL server
    password='230290',  # Password for the MySQL server
    host='localhost',            # Host address for the MySQL server (localhost in this case)
    database='sakila'            # Name of the database to connect to
)

# Checking if the connection to the database was successfully opened.
if cnx.open:
    print("Connection open")  # Prints confirmation if the connection is successful
else:
    print("Connection is not successfully open")  # Prints an error message if the connection fails


Connection open


The cursor is essential for interacting with the database. It enables sending SQL commands (e.g., SELECT, INSERT, UPDATE) and fetching query results.

In [17]:
# Creating a cursor object from the database connection.
# The cursor allows executing SQL commands and fetching results from the database.

cursor = cnx.cursor()

In [ ]:
# Drop the existing database (optional, for a fresh start)
# Uncomment the following lines if you need to reset the database entirely.
#query = "DROP DATABASE IF EXISTS ironhack_gambling"
#cursor.execute(query)
#print("Database dropped and ready to be recreated.")


In [19]:
# Creating a new database named 'ironhack_gambling' in the MySQL server.
query = ("""CREATE DATABASE IF NOT EXISTS ironhack_gambling""")  # SQL query to create the database
cursor.execute(query)  # Executes the query to create the database
print("Database created (or already exists).")

Database created (or already exists).


In [21]:
# Creating the 'account' table in the 'ironhack_gambling' database
query = """
CREATE TABLE IF NOT EXISTS ironhack_gambling.account(
    AccountNo VARCHAR(10) PRIMARY KEY,           -- Unique account number (length matches dataset)
    CustId INT NOT NULL UNIQUE,                  -- Unique customer ID (integer)
    AccountLocation VARCHAR(32) NOT NULL,        -- Account location (city/state)
    CurrencyCode VARCHAR(3) NOT NULL,            -- Currency code (ISO 4217, three-character code)
    DailyDepositLimit INT NOT NULL,              -- Daily deposit limit
    StakeScale FLOAT NOT NULL,                   -- Scaling factor for stakes
    SourceProd VARCHAR(10)                       -- Source product information (optional)
);
"""
cursor.execute(query)
print("Table 'account' created successfully.")


Table 'account' created successfully.


### Explanation of the `CREATE TABLE` Query for the `account` Table:

- **`AccountNo VARCHAR(52) PRIMARY KEY`**: A unique identifier for each account, stored as a string with a maximum length of 52 characters.
- **`CustId INT NOT NULL`**: The ID of the customer associated with the account, stored as an integer. This field cannot be null.
- **`AccountLocation VARCHAR(32) NOT NULL`**: Specifies the location of the account (e.g., "GIB"). Stored as a string with a maximum length of 32 characters.
- **`CurrencyCode VARCHAR(32) NOT NULL`**: The currency code for the account (e.g., "GBP", "USD"). Stored as a string with a maximum length of 32 characters.
- **`DailyDepositLimit INT NOT NULL`**: The maximum deposit limit allowed per day, stored as an integer.
- **`StakeScale FLOAT NOT NULL`**: A scale value for the stakes, stored as a floating-point number to allow decimals (e.g., `0.41`).
- **`SourceProd VARCHAR(32)`**: Indicates the source product associated with the account (e.g., "GM", "SB"). Stored as a string with a maximum length of 32 characters.

This query creates the `account` table if it does not already exist, ensuring no duplicate table creation.


In [23]:
# Creating the 'customer' table in the 'ironhack_gambling' database
query = """
CREATE TABLE IF NOT EXISTS ironhack_gambling.customer(
    CustID INT PRIMARY KEY,                      -- Unique customer ID
    AccountLocation VARCHAR(32) NOT NULL,       -- Location of the account
    Title VARCHAR(10),                          -- Title (e.g., Mr., Ms., Dr.)
    FirstName VARCHAR(64) NOT NULL,             -- First name of the customer
    LastName VARCHAR(64) NOT NULL,              -- Last name of the customer
    CreateDate DATE,                            -- Date when the account was created
    CountryCode VARCHAR(3),                     -- ISO Alpha-3 country code
    Language VARCHAR(8),                        -- Preferred language code (e.g., en, es, fr)
    Status CHAR(1),                             -- Account status (e.g., A for Active, I for Inactive)
    DateOfBirth DATE,                           -- Customer's date of birth
    Contact VARCHAR(32),                        -- Contact information (e.g., phone or email)
    CustomerGroup VARCHAR(32),                  -- Group category (e.g., Bronze, Silver, Gold)
    FOREIGN KEY (CustID) REFERENCES ironhack_gambling.account(CustId)  -- Ensures CustID matches in the 'account' table
)
"""
cursor.execute(query)
print("Table 'customer' created successfully.")


Table 'customer' created successfully.


### Explanation of the `CREATE TABLE` Query for the `customer` Table:

- **`CustID INT PRIMARY KEY`**: A unique integer identifier for each customer. This serves as the primary key.
- **`AccountLocation VARCHAR(32) NOT NULL`**: Specifies the location of the customer's account (e.g., "GIB").
- **`Title VARCHAR(32) NOT NULL`**: The title of the customer (e.g., "Mr", "Ms").
- **`FirstName VARCHAR(64) NOT NULL`**: The first name of the customer. Stored as a string with a maximum length of 64 characters.
- **`LastName VARCHAR(64) NOT NULL`**: The last name of the customer. Stored as a string with a maximum length of 64 characters.
- **`CreateDate VARCHAR(32)`**: The date the customer account was created. Stored as a string; it could be converted to a `DATE` type for stricter date handling.
- **`CountryCode VARCHAR(32)`**: The country code of the customer (e.g., "US", "UK").
- **`Language VARCHAR(32)`**: The customer's preferred language (e.g., "en").
- **`Status VARCHAR(32)`**: The status of the account (e.g., "A" for Active).
- **`DateOfBirth VARCHAR(32)`**: The customer's date of birth. Stored as a string; it could be converted to a `DATE` type for stricter date handling.
- **`Contact VARCHAR(32)`**: Indicates if the customer has provided contact information (e.g., "Y" for Yes, "N" for No).
- **`CustomerGroup VARCHAR(32)`**: The group to which the customer belongs (e.g., "Bronze", "Silver", "Gold").
- **`FOREIGN KEY (CustID) REFERENCES account(CustId)`**: This establishes a foreign key relationship between `CustID` in the `customer` table and `CustId` in the `account` table. It ensures that every `CustID` in `customer` corresponds to a valid `CustId` in `account`.

This query creates the `customer` table if it does not already exist. It includes a foreign key constraint to maintain referential integrity between the `customer` and `account` tables.


In [25]:
# Creating the 'betting' table in the 'ironhack_gambling' database
query = """
CREATE TABLE IF NOT EXISTS ironhack_gambling.betting(
    AccountNo VARCHAR(10) NOT NULL,             -- References primary key in the 'account' table
    BetDate DATE NOT NULL,                      -- Date of the bet (updated from TIMESTAMP to DATE)
    ClassId VARCHAR(32) NOT NULL,               -- Product classification ID
    CategoryId VARCHAR(64) NOT NULL,            -- Category ID for the product
    Source VARCHAR(10) NOT NULL,                -- Source of the betting activity
    BetCount INT NOT NULL,                      -- Number of bets placed
    Bet_Amt FLOAT NOT NULL,                     -- Amount of money bet (allows fractional values)
    Win_Amt FLOAT NOT NULL,                     -- Amount of money won
    Product VARCHAR(32) NOT NULL,               -- Name of the product
    AccountNo_1 VARCHAR(10),                    -- Optional reference to another account in the 'account' table
    Bet_Amt_1 FLOAT,                            -- Optional additional bet amount
    Product_1 VARCHAR(32),                      -- Optional additional product name
    AccountNo_2 VARCHAR(10),                    -- Optional reference to another account in the 'account' table
    Vegas DECIMAL(10, 2),                       -- Amount bet in Vegas (changed from FLOAT to DECIMAL for better precision)
    Sportsbook DECIMAL(10, 2),                  -- Amount bet in Sportsbook (changed from FLOAT to DECIMAL)
    Games DECIMAL(10, 2),                       -- Amount bet in Games (changed from FLOAT to DECIMAL)
    Casino DECIMAL(10, 2),                      -- Amount bet in Casino (changed from FLOAT to DECIMAL)
    Poker DECIMAL(10, 2),                       -- Amount bet in Poker (changed from FLOAT to DECIMAL)
    Bingo DECIMAL(10, 2),                       -- Amount bet in Bingo (changed from FLOAT to DECIMAL)
    Adjustments DECIMAL(10, 2),                 -- Adjustments to the betting amount (changed from FLOAT to DECIMAL)
    FOREIGN KEY (AccountNo) REFERENCES account(AccountNo),   -- Enforces referential integrity with 'account' table
    FOREIGN KEY (AccountNo_1) REFERENCES account(AccountNo), -- Optional reference to another account
    FOREIGN KEY (AccountNo_2) REFERENCES account(AccountNo)  -- Optional reference to another account
)
"""
cursor.execute(query)
print("Table 'betting' created successfully.")


Table 'betting' created successfully.


### Explanation of the `CREATE TABLE` Query for the `betting` Table:

- **`AccountNo VARCHAR(52) NOT NULL`**: A string that stores the account number associated with the bet. This field cannot be null.
- **`BetDate TIMESTAMP NOT NULL`**: The date and time of the bet, stored as a timestamp for precise date-time operations.
- **`ClassId VARCHAR(32) NOT NULL`**: A string identifier for the class of the bet.
- **`CategoryId VARCHAR(64) NOT NULL`**: A string identifier for the category of the bet.
- **`Source VARCHAR(32) NOT NULL`**: Indicates the source of the bet.
- **`BetCount INT NOT NULL`**: The number of bets placed, stored as an integer.
- **`Bet_Amt INT NOT NULL`**: The amount of money bet, stored as an integer. This assumes that the bet amount does not require decimal precision.
- **`Win_Amt FLOAT NOT NULL`**: The amount of money won, stored as a floating-point number to allow decimals.
- **`Product VARCHAR(32) NOT NULL`**: The product associated with the bet, such as "Vegas" or "Sportsbook".
- **`AccountNo_1 VARCHAR(32) NOT NULL`**: A secondary account number linked to the bet. This field is mandatory.
- **`Bet_Amt_1 FLOAT NOT NULL`**: The secondary bet amount, stored as a floating-point number.
- **`Product_1 VARCHAR(32)`**: A secondary product associated with the bet. This field is optional.
- **`AccountNo_2 VARCHAR(32)`**: A tertiary account number linked to the bet. This field is optional.
- **`Vegas FLOAT`**: Amount associated with bets categorized as "Vegas".
- **`Sportsbook FLOAT`**: Amount associated with bets categorized as "Sportsbook".
- **`Games FLOAT`**: Amount associated with bets categorized as "Games".
- **`Casino FLOAT`**: Amount associated with bets categorized as "Casino".
- **`Poker FLOAT`**: Amount associated with bets categorized as "Poker".
- **`Bingo FLOAT`**: Amount associated with bets categorized as "Bingo".
- **`Adjustments FLOAT`**: Adjustments made to the bet amount, stored as a floating-point number.
- **`FOREIGN KEY (AccountNo) REFERENCES account(AccountNo)`**: Establishes a foreign key relationship between `AccountNo` in the `betting` table and `AccountNo` in the `account` table. This ensures referential integrity by requiring that every `AccountNo` in `betting` matches an existing account.

This query ensures that the `betting` table is created if it doesn't already exist. The fields are designed to accommodate the dataset structure while enforcing a foreign key constraint to link the table with the `account` table. The foreign key relationship for `Product` has been removed since the values in `betting.Product` and `product.CLASSID` are not consistently aligned.


In [27]:
# Creating the 'product' table in the 'ironhack_gambling' database
query = ("""
CREATE TABLE IF NOT EXISTS
ironhack_gambling.product(
    CLASSID VARCHAR(32) PRIMARY KEY,
    CATEGORYID INT NOT NULL,
    Product VARCHAR(32) NOT NULL,
    sub_product VARCHAR(32) NOT NULL,
    description VARCHAR(64),
    bet_or_play TINYINT NOT NULL
)
""")
cursor.execute(query)
print("Table 'product' created successfully.")


Table 'product' created successfully.


### Explanation of the `CREATE TABLE` Query for the `product` Table:

- **`CLASSID VARCHAR(32) PRIMARY KEY`**: A string identifier for the class of the product. This field acts as the **primary key**, ensuring each product class is unique and serves as the main identifier for the table.
- **`CATEGORYID INT NOT NULL`**: The category ID of the product, stored as an integer. This field is mandatory.
- **`Product VARCHAR(32) NOT NULL`**: The main product name (e.g., "Sportsbook", "Vegas"). This field is mandatory.
- **`sub_product VARCHAR(32) NOT NULL`**: The sub-product name, providing more granularity about the product (e.g., "Football", "Vegas Bonus"). This field is mandatory.
- **`description VARCHAR(64)`**: A description of the product, such as its purpose or details. This field is optional.
- **`bet_or_play INT NOT NULL`**: Indicates whether the product involves betting (`1`) or playing (`0`), stored as an integer. This field is mandatory.

This query ensures the `product` table is created if it doesn't already exist. The **`CLASSID`** is defined as the **primary key**, ensuring that each product has a unique identifier for efficient data management and referential integrity.


In [29]:
# Creating the 'school' table in the 'ironhack_gambling' database
query = """
CREATE TABLE IF NOT EXISTS ironhack_gambling.school(
    school_id VARCHAR(32) PRIMARY KEY,          -- Unique identifier for each school
    school_name VARCHAR(64) NOT NULL,           -- Name of the school
    city VARCHAR(32) NOT NULL                   -- City where the school is located
)
"""
cursor.execute(query)
print("Table 'school' created successfully.")

Table 'school' created successfully.


### Explanation of the `CREATE TABLE` Query for the `school` Table:

- **`school_id_1 VARCHAR(32) PRIMARY KEY`**: A unique identifier for each school, stored as a string. This field serves as the primary key.
- **`school_name VARCHAR(32) NOT NULL`**: The name of the school, stored as a string with a maximum length of 32 characters. This field is mandatory.
- **`city_1 VARCHAR(32) NOT NULL`**: The city where the school is located, stored as a string. This field is mandatory.

This query creates the `school` table if it does not already exist. The structure is designed to store school-related data as per the provided dataset.


In [31]:
# Creating the 'student' table in the 'ironhack_gambling' database
query = """
CREATE TABLE IF NOT EXISTS ironhack_gambling.student(
    student_id VARCHAR(32) PRIMARY KEY,         -- Unique identifier for each student
    student_name VARCHAR(64) NOT NULL,          -- Name of the student
    city VARCHAR(32) NOT NULL,                  -- City where the student resides
    school_id VARCHAR(32),                      -- References the school the student is associated with
    GPA DECIMAL(3, 2) NOT NULL DEFAULT 0,       -- Student's Grade Point Average with precision (e.g., 3.95)
    FOREIGN KEY (school_id) REFERENCES school(school_id)  -- Enforces referential integrity with 'school' table
)
"""
cursor.execute(query)
print("Table 'student' created successfully.")

Table 'student' created successfully.


### Explanation of the `CREATE TABLE` Query for the `student` Table:

- **`student_id VARCHAR(32) PRIMARY KEY`**: A unique identifier for each student, stored as a string. This field serves as the primary key.
- **`student_name VARCHAR(32) NOT NULL`**: The name of the student, stored as a string with a maximum length of 32 characters. This field is mandatory.
- **`city VARCHAR(32) NOT NULL`**: The city where the student resides, stored as a string. This field is mandatory.
- **`school_id VARCHAR(32)`**: The school ID associated with the student, stored as a string. This field is optional and can contain NULL values.
- **`GPA FLOAT NOT NULL DEFAULT 0`**: The Grade Point Average of the student, stored as a floating-point number. This field is mandatory and defaults to 0 if no value is provided.

This query creates the `student` table if it does not already exist. The updated structure includes a foreign key constraint to maintain data consistency between the `student` and `school` tables and ensures `GPA` is stored as a numeric value for accurate calculations.


In [33]:
# Committing the changes made to the database
cnx.commit()


### Explanation of the `cnx.commit()` Statement:

- **Purpose**: The `cnx.commit()` command saves all the changes made during the session to the database.
- **Usage**: 
  - When creating tables, inserting data, or updating records in a database, these changes are initially staged and not permanently applied until a commit is executed.
  - This ensures that all operations are safely written to the database.
- **Importance**: Without calling `commit()`, the changes might not persist, especially if the connection to the database is closed.

In this case, it finalizes the creation of all the tables in the `ironhack_gambling` database.


In [43]:
# Print columns of the current table for debugging
print(f"Columns of the {data_title[j]} table:")
print(table.columns)


Columns of the school table:
Index(['school_id.1', 'school_name', 'city.1'], dtype='object')


In [45]:
# Loop through each table and insert data into its corresponding database table
for j in range(len(data)):
    table = data[j]  # Current Pandas DataFrame
    print(data_title[j])  # Print the name of the current table
    
    # Clean column names by replacing periods (or any unwanted characters) with underscores
    table.columns = table.columns.str.replace('.', '_', regex=True)
    
    # Print columns of the current table for debugging
    print(f"Columns of the {data_title[j]} table:")
    print(table.columns)
    
    columns = table.columns  # Extract column names
    parameter = ", ".join(list(columns))  # Create a comma-separated string of column names
    parameter = parameter.replace(".", "_")  # Replace '.' with '_' to match SQL column names
    param_2 = ("%s, " * len(columns))[:-2]  # Generate placeholders for each column (e.g., %s, %s, %s)
    query = f"INSERT IGNORE INTO ironhack_gambling.{data_title[j]}({parameter}) VALUES ({param_2})"  # SQL INSERT query with INSERT IGNORE
    
    # Loop through each row in the DataFrame
    for i in range(len(data[j])):
        row = list(table.iloc[i])  # Extract row data as a list
        
        # Remove NaN from the row and handle NULL for ClassId and Product columns
        row = [None if pd.isna(x) else x for x in row]  # Replace NaN with None
        
        # Handling NULL values for 'ClassId' column (assuming it's in the betting table)
        if 'ClassId' in table.columns:
            row[table.columns.get_loc('ClassId')] = row[table.columns.get_loc('ClassId')] or "Unknown"  # Replace NULL with "Unknown"
        
        # Handling NULL values for 'Product' column (assuming it's in the betting table)
        if 'Product' in table.columns:
            row[table.columns.get_loc('Product')] = row[table.columns.get_loc('Product')] or "Unknown"  # Replace NULL with "Unknown"
        
        print(row)  # Print the row being inserted (for debugging purposes)
        try:
            cursor.execute(query, row)  # Execute the query with the row data
        except pymysql.MySQLError as e:
            print(f"Error occurred: {e}")


account
Columns of the account table:
Index(['_________', '______', '_______________', '____________',
       '_________________', '__________', '__________'],
      dtype='object')
['00357DG   ', 3531845, 'GIB', 'GBP', 0, 1.0, 'GM']
Error occurred: (1054, "Unknown column '_________' in 'field list'")
['00497XG   ', 4188499, 'GIB', 'GBP', 0, 1.0, 'SB']
Error occurred: (1054, "Unknown column '_________' in 'field list'")
['00692VS   ', 4704925, 'GIB', 'USD', 0, 2.0, 'SB']
Error occurred: (1054, "Unknown column '_________' in 'field list'")
['00775SM   ', 2815836, 'GIB', 'USD', 0, 1.0, 'SB']
Error occurred: (1054, "Unknown column '_________' in 'field list'")
['00C017    ', 889782, 'GIB', 'GBP', 1500, 0.41, 'XX']
Error occurred: (1054, "Unknown column '_________' in 'field list'")
['00J381    ', 1191874, 'GIB', 'GBP', 500, 8.0, 'XX']
Error occurred: (1054, "Unknown column '_________' in 'field list'")
['01148BP   ', 1569944, 'GIB', 'GBP', 0, 8.0, 'XX']
Error occurred: (1054, "Unknown col

In [ ]:
# Commit the transaction after all inserts are done
cnx.commit()
